# NLP Fuzzy matching protype code

In [109]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.stem.porter import PorterStemmer
import csv

porter = PorterStemmer()

In [52]:
# Columns we are interested in
cols = ["Product", "Platform"]

#Read data
pdf = pd.read_csv("/Users/maxnethercott/Documents/data_science_projects/nlp_sentence_matching/fuzzy_prototype/ms_security_updates_csv.csv")
pdf = pdf[cols]
pdf.columns = pdf.columns.str.lower()
pdf = pdf.fillna("")

#Lower all pandas column values
pdf = pdf.apply(lambda x: x.astype(str).str.lower())

pdf['document'] = pdf[pdf.columns].agg('.'.join, axis=1)
pdf.drop_duplicates(inplace=True)


In [62]:
""" doc_list = pdf["document"].tolist()
print(len(doc_list))
tokenized_sents = [word_tokenize(i) for i in doc_list]

#Filter out stopwords and punctuation...
stop_words = set(nltk.corpus.stopwords.words("english"))

#Strip out stop words
out = [[w for w in x if not w in list(stop_words)] for x in tokenized_sents]
#Strip out punctuation
out = [[s.translate(str.maketrans("", "", string.punctuation)) for s in x] for x in out]

#Use porter stemmer to get root word meaning... Seems to have a positive effect on accuracy
out = [[porter.stem(word) for word in x if word != '']  for x in out]

#Detokenize
detokenized_sents = [TreebankWordDetokenizer().detokenize(i) for i in out]
data = list(set(detokenized_sents))
print(len(data)) """

151
151


In [116]:
#Filter out stopwords and punctuation...
stop_words = set(nltk.corpus.stopwords.words("english"))

# Tokenize, remove stop words, punctuation, stem words and de-tokenize
pdf["tokenize_sents"] = (pdf.apply(lambda row: word_tokenize(row["document"]), axis=1)
                            .apply(lambda x: [item for item in x if item not in list(stop_words)])
                            .apply(lambda x: [s.translate(str.maketrans("", "", string.punctuation)) for s in x])
                            .apply(lambda x: [porter.stem(word) for word in x if word != ''])
                            .apply(lambda x: TreebankWordDetokenizer().detokenize(x)))
                            
#Drop any duplicates after preprocessing
pdf = pdf.drop_duplicates(subset=["tokenize_sents"])
pdf.head()

,product,platform,document,tokenize_sents
0,microsoft sharepoint server 2019,,microsoft sharepoint server 2019.,microsoft sharepoint server 2019
1,microsoft sharepoint enterprise server 2013 se...,,microsoft sharepoint enterprise server 2013 se...,microsoft sharepoint enterpris server 2013 ser...
2,microsoft sharepoint enterprise server 2016,,microsoft sharepoint enterprise server 2016.,microsoft sharepoint enterpris server 2016
3,microsoft edge (chromium-based),,microsoft edge (chromium-based).,microsoft edg chromiumbas
8,microsoft biztalk esb toolkit 2.3,,microsoft biztalk esb toolkit 2.3.,microsoft biztalk esb toolkit 23


In [123]:
#Save data and training data

t_data = pdf["tokenize_sents"].tolist()
og_data = pdf["document"].tolist()

write = csv.writer(open ('og_data.csv', 'w'), delimiter=',', lineterminator='\n')
for x in og_data : write.writerow([x])

write = csv.writer(open ('training_data.csv', 'w'), delimiter=',', lineterminator='\n')
for x in t_data : write.writerow([x])

In [124]:
# Ready for model build
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(og_data)]
max_epochs = 50
vec_size = 150
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                min_count=1,
                dm =0,
                window =1,
                workers=4,
                sample = 0)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("final_d2v.model")
#with open("final_d2v.model", "wb+") as fname:
#    model.save(fname) 
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
Model Saved


In [126]:
def output_sentences(most_similar, og_data):
    for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(most_similar)//2), ('LEAST', len(most_similar) - 1)]:
      print(u'%s %s: %s\n' % (label, most_similar[index][1], og_data[int(most_similar[index][0])]))


model= Doc2Vec.load("d2v2.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("edge".lower())
test_data = [porter.stem(word) for word in test_data]
v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.dv.most_similar([v1])
print(similar_doc)

# to print similar sentences
output_sentences(similar_doc, og_data) 

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.dv['1'])

[('3', 0.9014874696731567), ('95', 0.8173137307167053), ('104', 0.8052164912223816), ('117', 0.7813312411308289), ('109', 0.7812954187393188), ('113', 0.77899569272995), ('102', 0.7626765966415405), ('103', 0.762440025806427), ('115', 0.7604480385780334), ('112', 0.7599497437477112)]
MOST 0.9014874696731567: microsoft edge (chromium-based).

SECOND-MOST 0.8173137307167053: microsoft edge for ios.

MEDIAN 0.77899569272995: microsoft edge (chromium-based) in ie mode.windows 10 version 1909 for x64-based systems

LEAST 0.7599497437477112: microsoft edge (chromium-based) in ie mode.windows 10 version 1909 for arm64-based systems

